In [ ]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
%%writefile front.py
import streamlit as st
import requests
import os
import io

def main():
    st.title("Chatbot Économique (RAG) - Squelette")


    if "session_id" not in st.session_state:
        st.session_state["session_id"] = None
    if "conversation_id" not in st.session_state:
        st.session_state["conversation_id"] = None
    if "conversations" not in st.session_state:
        st.session_state["conversations"] = {}
    if "messages" not in st.session_state:
        st.session_state["messages"] = {}


    # barre latérale : authentificairon

    with st.sidebar:
        st.subheader("Authentification")
        if st.session_state["session_id"] is None:
            username = st.text_input("Pseudo/Email", "")
            password = st.text_input("Mot de passe", "", type="password")

            if st.button("Se connecter"):
                if username and password:
                    # on simule un succès de connexion
                    st.session_state["session_id"] = "session_demo_id"
                    st.success("Connecté avec succès.")
                else:
                    st.error("Pseudo/Mot de passe invalide ou vide.")
        else:
            st.success("Vous êtes connecté.")
            if st.button("Se déconnecter"):
                # on reinit la session
                st.session_state["session_id"] = None
                st.session_state["conversation_id"] = None
                st.session_state["messages"] = {}
                st.session_state["conversations"] = {}

        st.write("---")


        # barre latérale : Upload PDF test

        if st.session_state["session_id"]:
            st.subheader("Ingestion de PDFs")
            uploaded_files = st.file_uploader("Choisissez un ou plusieurs PDF",
                                              type=["pdf"],
                                              accept_multiple_files=True)
            if st.button("Uploader et indexer"):
                if uploaded_files:
                    st.success("PDF(s) 'envoyé(s)' et 'ingéré(s)'. (Fictif)")
                else:
                    st.warning("Veuillez sélectionner au moins un PDF.")

        st.write("---")


        # barre latérale : liste des convo test

        if st.session_state["session_id"]:
            st.subheader("Vos conversations")

            # affichage de la liste des convo
            for cid, title in st.session_state["conversations"].items():
                if st.button(title, key=cid):
                    st.session_state["conversation_id"] = cid

            # bouton pour créer une nouvelle conversation
            if st.button("Nouvelle conversation"):
                new_cid = f"conv_{len(st.session_state['conversations']) + 1}"
                st.session_state["conversations"][new_cid] = f"Conversation {len(st.session_state['conversations'])}"
                st.session_state["conversation_id"] = new_cid
                # init de la liste de messages pour cette conv
                st.session_state["messages"][new_cid] = []
                st.success(f"Nouvelle conversation créée : {new_cid}")

    # zone principale : Chat

    if st.session_state["session_id"]:
        if st.session_state["conversation_id"] is None:
            st.info("Veuillez sélectionner ou créer une conversation dans la barre latérale.")
        else:
            conv_id = st.session_state["conversation_id"]
            st.subheader(f"Conversation en cours: {conv_id}")

            # l'historique de conversation
            if conv_id in st.session_state["messages"]:
                for msg in st.session_state["messages"][conv_id]:
                    role, content = msg
                    if role == "user":
                        st.markdown(f"**Vous**: {content}")
                    else:
                        st.markdown(f"**Assistant**: {content}")

            # barre pour poser une question
            user_query = st.text_input("Posez une question basée sur vos PDF")
            if st.button("Envoyer"):
                if user_query.strip():
                    #aj de la question dans l'historique
                    st.session_state["messages"][conv_id].append(("user", user_query))
                    # simule une réponse de l'assistant
                    assistant_reply = f"Réponse fictive à la question: '{user_query}'"
                    st.session_state["messages"][conv_id].append(("assistant", assistant_reply))
                    st.write(f"**Assistant**: {assistant_reply}")
                else:
                    st.warning("Veuillez saisir une question valide.")

if __name__ == "__main__":
    main()


Overwriting front.py


In [ ]:
from pyngrok import ngrok

# authtoken
ngrok.set_auth_token("letokken")


ngrok.kill()
public_url = ngrok.connect(8502)
public_url

<NgrokTunnel: "https://b4da-35-239-174-246.ngrok-free.app" -> "http://localhost:8502">

In [ ]:
!streamlit run front.py --server.port 8502 --server.headless true &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.239.174.246:8502

